<a href="https://colab.research.google.com/github/andrewk1/pytorch-deep-bayesian-bandits/blob/master/TrainDataEncodetextsBertParagraphEncoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!nvidia-smi

Fri Aug  9 02:46:19 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   30C    P8    26W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [4]:
!pip install regex requests
import torch
roberta = torch.hub.load('pytorch/fairseq', 'roberta.base')
roberta.eval() 
roberta.cuda()

     |████████████████████████████████| 655kB 2.8MB/s 
  Created wheel for regex: filename=regex-2019.6.8-cp36-cp36m-linux_x86_64.whl size=604152 sha256=653aae866aeb0a1f26d8f2d8007615f682c1a3c597017ba4680a7b31e9b484fa
  Stored in directory: /root/.cache/pip/wheels/35/e4/80/abf3b33ba89cf65cd262af8a22a5a999cc28fbfabea6b38473
Successfully built regex


Downloading: "https://github.com/pytorch/fairseq/archive/master.zip" to /root/.cache/torch/hub/master.zip
100%|██████████| 231160875/231160875 [00:08<00:00, 27313562.62B/s]


loading archive file http://dl.fbaipublicfiles.com/fairseq/models/roberta.base.tar.gz from cache at /root/.cache/torch/pytorch_fairseq/37d2bc14cf6332d61ed5abeb579948e6054e46cc724c7d23426382d11a31b2d6.ae5852b4abc6bf762e0b6b30f19e741aa05562471e9eb8f4a6ae261f04f9b350
extracting archive file /root/.cache/torch/pytorch_fairseq/37d2bc14cf6332d61ed5abeb579948e6054e46cc724c7d23426382d11a31b2d6.ae5852b4abc6bf762e0b6b30f19e741aa05562471e9eb8f4a6ae261f04f9b350 to temp dir /tmp/tmplk7rvyfn
| dictionary: 50264 types


1042301B [00:00, 2273765.11B/s]
456318B [00:00, 17587447.74B/s]


RobertaHubInterface(
  (model): RobertaModel(
    (decoder): RobertaEncoder(
      (sentence_encoder): TransformerSentenceEncoder(
        (embed_tokens): Embedding(50265, 768, padding_idx=1)
        (embed_positions): LearnedPositionalEmbedding(514, 768, padding_idx=1)
        (layers): ModuleList(
          (0): TransformerSentenceEncoderLayer(
            (self_attn): MultiheadAttention(
              (out_proj): Linear(in_features=768, out_features=768, bias=True)
            )
            (self_attn_layer_norm): LayerNorm(torch.Size([768]), eps=1e-05, elementwise_affine=True)
            (fc1): Linear(in_features=768, out_features=3072, bias=True)
            (fc2): Linear(in_features=3072, out_features=768, bias=True)
            (final_layer_norm): LayerNorm(torch.Size([768]), eps=1e-05, elementwise_affine=True)
          )
          (1): TransformerSentenceEncoderLayer(
            (self_attn): MultiheadAttention(
              (out_proj): Linear(in_features=768, out_features=7

In [0]:
def ParaGraphEncoderAverage(inputText):
    tokens = roberta.encode(inputText)
    tokens = tokens[0:512] #max input is 512
    all_layers = roberta.extract_features(tokens, return_all_hiddens=True)
    second_to_last = all_layers[-2]
    return torch.mean(second_to_last[0], 0).cpu().detach().numpy()

def ParaGraphEncoderMaxPooled(inputText):
    tokens = roberta.encode(inputText)
    tokens = tokens[0:512] #max input is 512
    all_layers = roberta.extract_features(tokens, return_all_hiddens=True)
    second_to_last = all_layers[-2]
    return torch.max(second_to_last[0], 0).cpu().detach().numpy()

def ParaGraphEncoderConcatMaxMinPooled(inputText):
    tokens = roberta.encode(inputText)
    tokens = tokens[0:512] #max input is 512
    all_layers = roberta.extract_features(tokens, return_all_hiddens=True)
    second_to_last = all_layers[-2]
    maxedd = torch.max(second_to_last[0], 0)
    minedd = torch.min(second_to_last[0], 0)
    return torch.cat( (maxedd[0], minedd[0]), 0 ).cpu().detach().numpy()

In [6]:
paragraphEmbeddingAveraged = ParaGraphEncoderAverage('Hello world!')
print(paragraphEmbeddingAveraged)

[ 8.73666927e-02 -1.99637026e-01  1.22691663e-02  2.06399903e-01
  1.24098711e-01  8.76930133e-02 -3.39047387e-02 -1.87423870e-01
 -1.17235914e-01  2.35696077e-01 -4.36867714e-01 -2.00869903e-01
 -5.86469769e-02 -4.52387542e-01 -3.40143472e-01 -9.85440910e-02
 -3.05399507e-01 -4.54312056e-01  4.33829017e-02  2.85150856e-01
 -3.25825699e-02 -2.03095600e-02 -9.98308286e-02 -3.05308878e-01
  3.34835619e-01 -1.61100924e-03  2.77123988e-01 -1.18258797e-01
  3.31543803e-01  2.86710232e-01  4.62458283e-02 -2.41641030e-01
  2.56850064e-01 -3.56712550e-01  1.53611138e-01  1.61433697e-01
  2.97572076e-01  2.75653452e-01  7.67790794e-01  1.20076492e-01
  2.96198040e-01 -5.88729501e-01 -4.82953200e-03  1.88496001e-02
  1.50541082e-01 -4.33758609e-02 -4.64460999e-01 -2.89933115e-01
  8.85296986e-02 -1.65153906e-01  1.42797083e-02  1.43214747e-01
 -1.69136655e-02  1.76706519e-02  1.63483605e-01  3.05359930e-01
 -2.35360309e-01  5.56727588e-01 -9.95012075e-02 -3.16942036e-02
 -1.17717467e-01  4.33179

In [7]:
# paragraphEmbeddingMaxPooled = ParaGraphEncoderMaxPooled('Hello world!')
# print(paragraphEmbeddingMaxPooled[0])

AttributeError: ignored

In [0]:
# paragraphEmbeddingMaxMinConcat = ParaGraphEncoderConcatMaxMinPooled('Hello world!')
# print(paragraphEmbeddingMaxMinConcat)

In [0]:
import requests

def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

In [0]:
#Download yelp data

download_file_from_google_drive('0Bz8a_Dbh9QhbNUpYQ2N3SGlFaDg', 'yelp.tar.gz')

In [0]:
import tarfile
tf = tarfile.open("yelp.tar.gz")
tf.extractall()

In [0]:
import os

In [12]:
os.listdir('yelp_review_polarity_csv')

['test.csv', 'train.csv', 'readme.txt']

In [0]:
import pandas as pd

In [0]:
df = pd.read_csv('yelp_review_polarity_csv/train.csv', header =None, nrows = 62000)

In [29]:
df.head()

,0,1
0,1,"Unfortunately, the frustration of being Dr. Go..."
1,2,Been going to Dr. Goldberg for over 10 years. ...
2,1,I don't know what Dr. Goldberg was like before...
3,1,I'm writing this review to give you a heads up...
4,2,All the food is great here. But the best thing...


In [30]:
df.shape

(62000, 2)

In [0]:
df.columns = ['label', 'text']

In [32]:
df.head()

,label,text
0,1,"Unfortunately, the frustration of being Dr. Go..."
1,2,Been going to Dr. Goldberg for over 10 years. ...
2,1,I don't know what Dr. Goldberg was like before...
3,1,I'm writing this review to give you a heads up...
4,2,All the food is great here. But the best thing...


In [0]:
import numpy as np

In [0]:
df["averagePooled"] = ''


In [35]:
df.shape[0]

62000

In [36]:
df.loc[8]['text']

"Before I finally made it over to this range I heard the same thing from most people - it's just fine to go work on your swing. I had such a low expectation I was pleasantly surprised. \\n\\nIt's a fairly big range - if you are familiar with Scally's in Moon, it seems like it has almost as many tees, though its not nearly as nice a facility. \\n\\nThe guys in the pro shop were two of the friendlier guys I've come across at ranges or at courses. Yards were indeed marked and there are some targets to aim for, and even some hazards to aim away from. \\n\\nA big red flag to me was the extra charge ($3) to hit off the grass. I am no range expert, but this is the 4th one I've been to and the first I've seen of that sort of nickel and diming....\\n\\nPrice for the golf balls was reasonable and I do plan to be back every week until they close up in October for the season. Hopefully, since its for sale, it will reopen as a golf facility again."

In [37]:
len(ParaGraphEncoderAverage(df.loc[0]['text']) )

768

In [0]:
df.at[0, 'averagePooled'] = ParaGraphEncoderAverage(df.loc[0]['text'] )

In [39]:
df.head()

,label,text,averagePooled
0,1,"Unfortunately, the frustration of being Dr. Go...","[0.120301716, 0.10964332, -0.0901535, -0.18390..."
1,2,Been going to Dr. Goldberg for over 10 years. ...,
2,1,I don't know what Dr. Goldberg was like before...,
3,1,I'm writing this review to give you a heads up...,
4,2,All the food is great here. But the best thing...,


In [0]:
df['averagePooledValues'] = df.text.apply(ParaGraphEncoderAverage)